# Lemniscate Snapshot Utilities Demo

This notebook builds a lemniscate (∞) factor graph, runs a single PropFlow engine execution with snapshots enabled, and exercises the snapshot utilities and visualisation helpers end-to-end.

In [ ]:
from __future__ import annotations

from dataclasses import asdict
from pathlib import Path
import json

import matplotlib.pyplot as plt
import numpy as np

from analyzer.reporting import SnapshotAnalyzer, AnalysisReport
from analyzer.reporting.snapshot_parser import from_engine_snapshot_manager
from analyzer.snapshot_visualizer import SnapshotVisualizer
from propflow.bp.engine_base import BPEngine
from propflow.configs import create_random_int_table
from propflow.snapshots.types import SnapshotsConfig
from propflow.snapshots.utils import (
    latest_snapshot,
    latest_jacobians,
    latest_cycles,
    latest_winners,
    get_snapshot,
)
from propflow.utils.fg_utils import FGBuilder


In [ ]:
np.random.seed(7)

RUN_DIR = Path("results/notebooks/lemniscate_all_utils")
RUN_DIR.mkdir(parents=True, exist_ok=True)

snapshot_cfg = SnapshotsConfig(
    compute_jacobians=True,
    compute_block_norms=True,
    compute_cycles=True,
    include_detailed_cycles=True,
    compute_numeric_cycle_gain=False,
    retain_last=None,
    save_each_step=False,
)

graph = FGBuilder.build_lemniscate_graph(
    num_vars=8,
    domain_size=3,
    ct_factory=create_random_int_table,
    ct_params={"low": 1, "high": 7},
)

engine = BPEngine(
    factor_graph=graph,
    snapshots_config=snapshot_cfg,
    use_bct_history=True,
)


In [ ]:
MAX_STEPS = 10
engine.run(max_iter=MAX_STEPS)
manager = engine._snapshot_manager
captured_steps = list(range(len(engine.history.step_costs)))
print(f"Captured {len(captured_steps)} steps; final step = {captured_steps[-1]}")


In [ ]:
latest = latest_snapshot(engine)
print("Latest snapshot step:", latest.data.step)

jac = latest_jacobians(engine)
print("Jacobian blocks available:", jac is not None)
if jac and jac.block_norms:
    print("Block norms:", jac.block_norms)

cycles = latest_cycles(engine)
print("Cycle metrics:", cycles.num_cycles if cycles else None)

winners = latest_winners(engine)
print("Winner entries:", len(winners) if winners else 0)

first_step = captured_steps[0]
first_rec = get_snapshot(engine, first_step)
print("First step Q messages:", list(first_rec.data.Q.keys())[:3])


In [ ]:
last_step = captured_steps[-1]
step_dir = manager.save_step(last_step, RUN_DIR, save=True)
print("Persisted step directory:", step_dir)
meta = json.loads((step_dir / "meta.json").read_text())
meta["context"]


In [ ]:
typed_records = [
    from_engine_snapshot_manager(get_snapshot(engine, step))
    for step in captured_steps
]
raw_records = [asdict(record) for record in typed_records]
print("Total records:", len(raw_records))
raw_records[0].keys()


In [ ]:
analyzer = SnapshotAnalyzer.from_records(typed_records)
report = AnalysisReport(analyzer)
summary = report.to_json(step_idx=captured_steps[-1])
summary_keys = sorted(summary.keys())
print("Analysis report keys:", summary_keys)
summary["block_norms"]


In [ ]:
visualizer = SnapshotVisualizer.from_object(raw_records)
print("Variables:", visualizer.variables())
series = visualizer.argmin_series()
print("Series tracked for", len(series), "variables")

visualizer.plot_argmin_per_variable(vars_filter=visualizer.variables()[:6], figsize=(10, 12))
plt.show()
